In [1]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import math

import pandas as pd

#### Two new columns were added to the test & train dataset - SignWidth & SignHeight. The columns are result of dividing  SignWidth_old & SignHeight_old with their diagonals. 

In [4]:
train = pd.read_csv("dataset/train.csv")
test = pd.read_csv("dataset/test.csv")

In [5]:
train.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth_old,SignHeight_old,SignFacing (Target),SignWidth,SignHeight
0,2c9180975a056a64015a1e0a52e57021,Rear,195,1.02,46,45,Rear,0.714834,0.699294
1,2c9180975a056a64015a1e17b32171e4,Rear,203,1.09,59,54,Rear,0.737673,0.675158
2,2c9180975a056a64015a1de4deb16bd5,Front,26,0.96,104,108,Front,0.693642,0.720320
3,2c9180975a056a64015a1de4deb16bdd,Rear,199,0.81,38,47,Rear,0.628722,0.777630
4,2c9180975a056a64015a1de4deb16bd6,Rear,208,0.93,54,58,Rear,0.681419,0.731894


In [6]:
test.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth_old,SignHeight_old,SignWidth,SignHeight
0,2c9180975a056a64015a1e10d3f270fe,Right,67,0.63,107,169,0.534933,0.844894
1,2c9180975a056a64015a1de4deb16bdc,Front,16,0.88,61,69,0.662340,0.749204
2,2c9180975a056a64015a1e0e70ea70ce,Right,44,1.15,232,202,0.754185,0.656661
3,2c9180975a056a64015a1dfed0c46ec6,Right,50,1.10,137,125,0.738719,0.674014
4,2c9180975a056a64015a1dfed0c46ec7,Front,30,0.95,99,104,0.689481,0.724304


In [7]:
train['DetectedCamera'].value_counts()

Front    10910
Right    10516
Left      9298
Rear      7761
Name: DetectedCamera, dtype: int64

In [8]:
#encode as integer
mapping = {'Front':0, 'Right':1, 'Left':2, 'Rear':3}
train = train.replace({'DetectedCamera':mapping})
test = test.replace({'DetectedCamera':mapping})

In [9]:
#renaming column
train.rename(columns = {'SignFacing (Target)': 'Target'}, inplace=True)

In [10]:
#encode Target Variable based on sample submission file
mapping = {'Front':0, 'Left':1, 'Rear':2, 'Right':3}
train = train.replace({'Target':mapping})

In [11]:
#target variable
y_train = train['Target']
test_id = test['Id']
train_id = train['Id']

In [12]:
#drop columns
train.drop(['Target','Id'], inplace=True, axis=1)
test.drop('Id',inplace=True,axis=1)

In [13]:
train.drop(['SignWidth_old','SignHeight_old'], inplace=True, axis=1)
test.drop(['SignWidth_old','SignHeight_old'], inplace=True, axis=1)

In [14]:
train.head()

,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight
0,3,195,1.02,0.714834,0.699294
1,3,203,1.09,0.737673,0.675158
2,0,26,0.96,0.693642,0.720320
3,3,199,0.81,0.628722,0.777630
4,3,208,0.93,0.681419,0.731894


In [17]:
test.head()

,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight
0,1,67,0.63,0.534933,0.844894
1,0,16,0.88,0.662340,0.749204
2,1,44,1.15,0.754185,0.656661
3,1,50,1.10,0.738719,0.674014
4,0,30,0.95,0.689481,0.724304


In [18]:
y_train.head()

0    2
1    2
2    0
3    2
4    2
Name: Target, dtype: int64

In [24]:
# learningRate = 0.001
numIters = 1001
displayStep = 20

In [25]:
# placeholders and variables
X = tf.placeholder(tf.float32,[None,5])
Y_ = tf.placeholder(tf.float32,[None,4])
lr = tf.placeholder(tf.float32)

# weights and biases
W1 = tf.Variable(tf.truncated_normal([5,90], stddev=0.1))
b1 = tf.Variable(tf.ones([90])/10)

W2 = tf.Variable(tf.truncated_normal([90,180], stddev=0.1))
b2 = tf.Variable(tf.ones([180])/10)

W3 = tf.Variable(tf.truncated_normal([180,60],stddev=0.1))
b3 = tf.Variable(tf.ones([60])/10)

W4 = tf.Variable(tf.truncated_normal([60,4],stddev=0.1))
b4 = tf.Variable(tf.ones([4])/10)

In [26]:
# model
Y1 = tf.nn.relu(tf.matmul(X,W1) + b1)
Y2 = tf.nn.relu(tf.matmul(Y1,W2) + b2)
Y3 = tf.nn.relu(tf.matmul(Y2,W3) + b3)
Ylogits = tf.matmul(Y3,W4) + b4
Y = tf.nn.softmax(Ylogits)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_, logits=Ylogits))*100
train_step = tf.train.AdamOptimizer(lr).minimize(loss)

correct_prediction = tf.equal(tf.argmax(Y,1),tf.argmax(Y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# X_train, X_val, y_train, y_val = train_test_split(train, y_train, train_size=0.65, random_state = 1)

In [27]:
# learning rate decay
max_learning_rate = 0.003
min_learning_rate = 0.0001
decay_speed = 2000.0

# testDict = {X: X_val.values ,Y_: np.eye(4)[y_val] }
for i in range(numIters):
    learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)
    feedDict = {X:train.values ,Y_:np.eye(4)[y_train], lr:learning_rate}
    
    sess.run(train_step,feed_dict=feedDict)
    
    if i%displayStep == 0:
        acc,cost = sess.run([accuracy,loss], feed_dict=feedDict)
        print(i, ": Train accuracy:" + str(acc) + " loss: " + str(cost))
        
#         test_acc,test_cost = sess.run([accuracy,loss], feed_dict=testDict)
#         print("Test accuracy:" + str(test_acc) + " loss: " + str(test_cost))

0 : Train accuracy:0.407483 loss: 100.253
20 : Train accuracy:0.914226 loss: 61.2236
40 : Train accuracy:0.646434 loss: 56.5615
60 : Train accuracy:0.918176 loss: 34.8936
80 : Train accuracy:0.917942 loss: 32.6006
100 : Train accuracy:0.918072 loss: 30.7015
120 : Train accuracy:0.918851 loss: 27.5273
140 : Train accuracy:0.932935 loss: 23.1704
160 : Train accuracy:0.93374 loss: 20.9944
180 : Train accuracy:0.935092 loss: 19.1348
200 : Train accuracy:0.923009 loss: 31.4137
220 : Train accuracy:0.934962 loss: 22.673
240 : Train accuracy:0.93504 loss: 20.2981
260 : Train accuracy:0.93504 loss: 18.8182
280 : Train accuracy:0.935767 loss: 17.8738
300 : Train accuracy:0.941042 loss: 17.1148
320 : Train accuracy:0.944212 loss: 16.446
340 : Train accuracy:0.940366 loss: 17.555
360 : Train accuracy:0.935767 loss: 25.4606
380 : Train accuracy:0.935403 loss: 18.6482
400 : Train accuracy:0.942367 loss: 16.7173
420 : Train accuracy:0.944862 loss: 16.2006
440 : Train accuracy:0.946914 loss: 15.6148


In [ ]:
# acc = sess.run(correct_prediction,feed_dict={X:train.values, Y_:np.eye(4)[y_train]}) # to test train accuracies

In [28]:
pred = sess.run(Y,feed_dict={X: test.values})

In [30]:
#write submission file and submit
columns = ['Front','Left','Rear','Right']
sub = pd.DataFrame(data=pred, columns=columns)
sub['Id'] = test_id
sub = sub[['Id','Front','Left','Rear','Right']]
sub.to_csv("submission/sub_nn.csv", index=False)